# 가설검정의 이해

### 실습 데이터셋 준비

In [ ]:
# 관련 라이브러리를 호출합니다.
import os, joblib
import numpy as np
import pandas as pd
from scipy import stats

In [ ]:
# 현재 작업 경로를 확인합니다.
os.getcwd()

In [ ]:
# data 폴더로 작업 경로를 변경합니다.
os.chdir('../data')

In [ ]:
# 현재 작업 경로에 포함된 폴더명과 파일명을 출력합니다.
os.listdir()

In [ ]:
# 압축 파일을 읽고 데이터프레임 df를 생성합니다.
df = joblib.load(filename = 'Used_Cars_Price.z')

In [ ]:
# df의 정보를 확인합니다.
df.info()

In [ ]:
# df의 처음 5행을 출력합니다.
df.head()

### 공분산

In [ ]:
# 두 연속형 변수의 공분산을 반환합니다.
df['Age'].cov(df['Price'])

In [ ]:
# df에 포함된 모든 연속형 변수의 분산-공분산 행렬을 반환합니다.
df.cov().round(2)

### 상관계수

In [ ]:
# 두 연속형 변수의 피어슨 상관계수를 반환합니다.
df['Age'].corr(df['Price'])

In [ ]:
# df에 포함된 모든 연속형 변수의 피어슨 상관계수 행렬을 반환합니다.
df.corr().round(2)

### 피어슨 상관분석

In [ ]:
# 관련 라이브러리를 호출합니다.
import pingouin as pg

In [ ]:
# Age와 Price의 피어슨 상관분석을 실행합니다.
pg.corr(x = df['Age'], y = df['Price'])

In [ ]:
# 연속형 입력변수명을 리스트로 생성합니다.
cols = ['Age', 'KM', 'HP', 'CC', 'Doors', 'Weight']

In [ ]:
# 연속형 입력변수와 Price의 피어슨 상관분석 유의확률만 데이터프레임으로 반환합니다.
df[cols].apply(func = lambda x: pg.corr(x = x, y = df['Price'])['p-val'])

### t-검정: MetColor

In [ ]:
# 정규성 검정을 실행합니다.
pg.normality(data = df, 
             dv = 'Price', 
             group = 'MetColor', 
             method = 'shapiro')

In [ ]:
# (정규성 가정 만족) MetColor의 범주별 Price의 등분산성 검정을 실행합니다.
pg.homoscedasticity(data = df, 
                    dv = 'Price', 
                    group = 'MetColor', 
                    method = 'bartlett')

In [ ]:
# 범주형 변수별 연속형 변수를 시리즈로 생성합니다.
s0 = df['Price'][df['MetColor'].eq(0)]
s1 = df['Price'][df['MetColor'].eq(1)]

In [ ]:
# 이분산 가정된 t-검정을 실행합니다.
pg.ttest(x = s0, y = s1, correction = True)

In [ ]:
# (정규성 가정 불만족) 맨-휘트니 U 검정을 실행합니다.
pg.mwu(x = s0, y = s1)

### t-검정: Automatic

In [ ]:
# 정규성 검정을 실행합니다.
pg.normality(data = df, 
             dv = 'Price', 
             group = 'Automatic', 
             method = 'shapiro')

In [ ]:
# (정규성 가정 만족) Automatic의 범주별 Price의 등분산성 검정을 실행합니다.
pg.homoscedasticity(data = df, 
                    dv = 'Price', 
                    group = 'Automatic', 
                    method = 'bartlett')

In [ ]:
# 범주형 변수별 연속형 변수를 시리즈로 생성합니다.
s0 = df['Price'][df['Automatic'].eq(0)]
s1 = df['Price'][df['Automatic'].eq(1)]

In [ ]:
# 등분산 가정된 t-검정을 실행합니다.
pg.ttest(x = s0, y = s1, correction = False)

In [ ]:
# (정규성 가정 불만족) 맨-휘트니 U 검정을 실행합니다.
pg.mwu(x = s0, y = s1)

### 분산분석: FuelType

In [ ]:
# 정규성 검정을 실행합니다.
pg.normality(data = df, 
             dv = 'Price', 
             group = 'FuelType', 
             method = 'shapiro')

In [ ]:
# (정규성 가정 만족) FuelType의 범주별 Price의 등분산성 검정을 실행합니다.
pg.homoscedasticity(data = df, 
                    dv = 'Price', 
                    group = 'FuelType', 
                    method = 'bartlett')

In [ ]:
# 등분산 가정된 분산분석을 실행합니다.
pg.anova(data = df, dv = 'Price', between = 'FuelType')

In [ ]:
# 이분산 가정된 분산분석을 실행합니다.
pg.welch_anova(data = df, dv = 'Price', between = 'FuelType')

In [ ]:
# (정규성 가정 불만족) 크루스칼-왈리스 순위합 검정을 실행합니다.
pg.kruskal(data = df, dv = 'Price', between = 'FuelType')

In [ ]:
# Tukey 방식으로 사후분석을 실행합니다.
pg.pairwise_tukey(data = df, dv = 'Price', between = 'FuelType')

### 교차분석

In [ ]:
# 두 범주형 변수로 교차테이블을 출력합니다.
pd.crosstab(index = df['MetColor'], columns = df['Automatic'])

In [ ]:
# 카이제곱 검정을 실행합니다.
pg.chi2_independence(data = df, x = 'MetColor', y = 'Automatic')

### 변수 제거 및 압축 파일로 저장

In [ ]:
# 가설검정 결과를 반영하여 CC 및 Automatic을 삭제합니다.
df = df.drop(labels = ['CC', 'Automatic'], axis = 1)

In [ ]:
# FuelType이 'CNG'인 건수가 매우 적고 평균 차이가 없으므로 삭제합니다.
df = df[df['FuelType'].ne('CNG')]

In [ ]:
# df의 정보를 확인합니다.
df.info()

In [ ]:
# df를 압축 파일로 저장합니다.
joblib.dump(value = df, filename = 'Used_Cars_Price_Prep.z')

## End of Document